In [77]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sc_util import sc_levels, plots_out_dir
from parse_case_sheet import get_clean_df

In [3]:
coarse_injection_site_mapping = {
    'VISp': 'VIS',  'PTLp': 'PTL', 'VISal': 'VIS', 'VISam': 'VIS', 
    'VISl': 'VIS',  'SSp-tr': 'SSp', 'ACAd': 'ACA', 'RSPv': 'RSP', 
    'ACAv': 'ACA', 'RSPagl': 'RSP', 'VISpm': 'VIS', 'RSPd': 'RSP',  
    'MOp': 'MO', 'ORBl': 'ORB', 'MOs': 'MO', 'SSp-ul': 'SSp', 'PL': 'PL', 
    'ORBvl': 'ORB', 'ORBm': 'ORB', 'SSp-ll': 'SSp', 'SSp-m': 'SSp', 
    'SSp-bfd': 'SSp', 'SSp-n': 'SSp', 'SSs': 'SSs', 'AUDd': 'AUD', 'AUDp': 'AUD', 'AUDv': 'AUD', 
    'TEa': 'TE', 'ILA': 'ILA', 'SNr': 'SN', 'SNr/SNc': 'SN', 'SC2': 'SC',
    'SC3': 'SC', 'SC1': 'SC', 'SC4': 'SC', 'SC3/PAG': 'SC', 'SC.l': 'SC', 'SC.cl/': 'SC', 'SC.cl': 'SC',
    'SC.cm': 'SC', 'SC.m': 'SC'
}

In [75]:
def get_polar_coordinates(ara_level):
    df_clean = get_clean_df()
    csv_path = 'polar_coordinates/polar_coordinates_ara_{}.csv'.format(str(ara_level).zfill(3))
    if not os.path.isfile(csv_path):
        return None
    df_polar = pd.read_csv(csv_path, index_col=0)
    df_polar = df_polar.rename(columns={ 'case': 'Case ID', 'channel': 'Channel' })
    # to same type as 'Channel' of df_clean
    df_polar['Channel'] = df_polar['Channel'].astype(str)
    # convert from radians to degrees
    df_polar['theta'] = df_polar['theta'] * (180 / np.pi)
    df_polar = df_polar.merge(df_clean[['Tracers', 'Cortex name with regional identifier', 
                                        'Actual Anatomical/ Injection Site Primary', 'Actual Anatomical/ Injection Site']], 
                              how='left', on=['Case ID', 'Channel'])
    return df_polar

In [58]:
def set_plot_ax(ax, title):
    # flip x axis to have descending xticks    
    ax.set_xlim(ax.get_xlim()[1], ax.get_xlim()[0])    
    ax.legend()   
    _ = ax.set_title(title)    

In [76]:
primary_injection_sites = ['RSPv', 'VISp', 'ACAd', 'MOp', 'SSp-tr', 'SSp-ul', 'SSp-bfd', 'SSp-m', 'SSp-ll', 'SSp-n']

def plot_polar_coordinates(ara_level):
    df_polar = get_polar_coordinates(ara_level)
    if df_polar is None:
        return
    df_polar_primary_injection_sites = {}
    for primary_injection_site in primary_injection_sites:
        df_polar_primary_injection_sites[primary_injection_site] = df_polar.loc[df_polar['Actual Anatomical/ Injection Site Primary'] == primary_injection_site, :]
    # plot all cases from primary injection sites
    fig, ax = plt.subplots(figsize=(20, 10))
    for primary_injection_site, df_polar_primary_injection_site in df_polar_primary_injection_sites.items():
        sns.distplot(df_polar_primary_injection_site['theta'], hist=False, label=primary_injection_site, ax=ax)
    title = 'all_cases_RSPv_VISp_ACAd_MOp_SSp_ARA_{}'.format(str(ara_level).zfill(3))
    set_plot_ax(ax, title)  
    plt.savefig(os.path.join(plots_out_dir(), '{}.png'.format(title)))
    # plot individual cases for each primary injection site
    df_clean = get_clean_df()
    for primary_injection_site, df_polar_primary_injection_site in df_polar_primary_injection_sites.items():
        fig, ax = plt.subplots(figsize=(20, 10))
        # unique case id and channel combination for rows with primary_injection_site
        df_case_channel = df_polar_primary_injection_site[['Case ID', 'Channel']].drop_duplicates()
        # (case_id, channel) tuple
        case_id_channels = list(zip(df_case_channel['Case ID'], df_case_channel['Channel']))
        for case_id_channel in case_id_channels:
            case_id, channel = case_id_channel
            cortex_name = df_clean.loc[case_id_channel, 'Cortex name with regional identifier']
            selection = (df_polar_primary_injection_site['Case ID'] == case_id) & (df_polar_primary_injection_site['Channel'] == channel)
            sns.distplot(df_polar_primary_injection_site.loc[selection, 'theta'], hist=False, label='{}_ch{}_{}'.format(case_id, channel, cortex_name), ax=ax)
        title = 'individual_cases_{}_ara_{}'.format(primary_injection_site, str(ara_level).zfill(3))    
        set_plot_ax(ax, title)  
        plt.savefig(os.path.join(plots_out_dir(), '{}.png'.format(title)))

In [ ]:
for sc_level in sc_levels:
    plot_polar_coordinates(sc_level)